In [1]:
# This is a notebook to summarize all different models.
# THis notebook was not run


## Single Encoder
# encoder_dense_layers_trial = [[8], [10], [12], [14], [16], [18] ,[20]]
# decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
# bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [2]:
#Disable Warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

In [3]:
FILE1 = 'SB-001_normalized_AM.csv'
FILE2 = 'SB-001_normalized_PM.csv'
NO_OF_COLUMNS = 24

In [4]:
#pip install tf2onnx
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
plt.ioff()
import cv2
import IPython
from tqdm import tqdm
import time
import os
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', message=r'.*Consider either turning off auto-sharding.*')

In [5]:
run_summary = []

In [6]:
def generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers):

    print(encoder_dense_layers, bottle_neck, decoder_dense_layers)
    # Load Data
    df_am = pd.read_csv(FILE1)
    df_pm = pd.read_csv(FILE2)
    combined_df = pd.concat([df_am, df_pm])

    columns_needed = ['y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in', 'tempdiffin', 'humidiffin', 'pm25diffin',
                      'pm10', 'pm25', 'o3', 'no2', 'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp',
                      'seap', 'solrhr', 'solramnt', 'grdt', 'class']
    combined_df = combined_df.filter(columns_needed)
    df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Drop NaN
    df = df.dropna()

    # Minotity Split
    class_counts = df['class'].value_counts()
    minority_class = df[df['class'] == 0]
    majority_class = df[df['class'] == 1]

    # Removing class column from minority before augmentation
    X = minority_class.drop('class', axis=1)
    y = minority_class['class']

    # Saving minority as X_train
    X_train = X

    # Define the Input shape
    INPUT_SHAPE = X_train.shape[1]
    FILE_NAME = f"L{INPUT_SHAPE}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"
    print("\n" + FILE_NAME + "\n")
    
    def build_autoencoder(input_shape, **kwargs):

        # Autoencoder model construction cod

      """
      Build an autoencoder model with the given configuration.

      Args:
          input_shape (int): The shape of the input data.
          **kwargs: Additional keyword arguments for configuring the autoencoder.

      Keyword Args:
          encoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the encoder.
                                      Default: []
          bottle_neck (int): The number of units in the bottleneck layer. Default: input_shape // 2
          decoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the decoder.
                                      Default: []
          decoder_activation (str): The activation function for the decoder output layer. Default: 'sigmoid'

      Returns:
          tuple: A tuple containing the autoencoder, encoder, and decoder models.

      Example:
          INPUT_SHAPE = 27
          encoder_dense_layers = [20, 18]  # Specify the number of units for each dense layer in the encoder
          bottle_neck = 16
          decoder_dense_layers = [18, 20]  # Specify the number of units for each dense layer in the decoder

          autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                            bottle_neck=bottle_neck, decoder_dense_layers=decoder_dense_layers)
          encoder.summary()
          decoder.summary()
          autoencoder.summary()
      """

      # Default parameter values
      encoder_dense_layers = kwargs.get('encoder_dense_layers', [])
      bottle_neck = kwargs.get('bottle_neck', input_shape // 2)
      decoder_dense_layers = kwargs.get('decoder_dense_layers', [])
      decoder_activation = kwargs.get('decoder_activation', 'sigmoid')

      # Autoencoder Model
      encoder_input = keras.Input(shape=(input_shape,), name="encoder")
      x = keras.layers.Flatten()(encoder_input)

      # Encoder Dense Layers
      for units in encoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      encoder_output = keras.layers.Dense(bottle_neck, activation="relu")(x)
      encoder = keras.Model(encoder_input, encoder_output, name="encoder")

      # Decoder Model
      decoder_input = keras.Input(shape=(bottle_neck,), name="decoder")
      x = decoder_input

      # Decoder Dense Layers
      for units in decoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      decoder_output = keras.layers.Dense(input_shape, activation=decoder_activation)(x)
      decoder = keras.Model(decoder_input, decoder_output, name="decoder")

      # Autoencoder Model
      autoencoder_input = keras.Input(shape=(input_shape,), name="input")
      encoded = encoder(autoencoder_input)
      decoded = decoder(encoded)
      autoencoder = keras.Model(autoencoder_input, decoded, name="autoencoder")

      return autoencoder, encoder, decoder

    autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                      bottle_neck=bottle_neck,
                                                      decoder_dense_layers=decoder_dense_layers)
    opt = keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(opt, loss="mse")

    print("Training model:", FILE_NAME)
    history = autoencoder.fit(X_train, X_train, epochs=200, batch_size=16, validation_split=0.25, verbose=0)
    print("Training Complete:")
    
    # Extract the loss values
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Print the last epoch's loss values
    last_epoch_loss = loss[-1]
    last_epoch_val_loss = val_loss[-1]
    print("Last epoch loss:", last_epoch_loss)
    print("Last epoch validation loss:", last_epoch_val_loss)

    # Saving history
    with open(FILE_NAME + '_history.pickle', 'wb') as file:
        pickle.dump(history.history, file)

    # Visualize losses *(MSE)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(FILE_NAME + 'Loss.png')
    #plt.show()

    # Generate synthetic data
    num_samples = len(X_train)
    input_data = np.random.normal(size=(num_samples, INPUT_SHAPE))
    generated_data = autoencoder.predict(input_data)
    reshaped_data = generated_data.reshape(num_samples, -1)
    df_generated = pd.DataFrame(reshaped_data, columns=X_train.columns)

    # Plot correlation matrices
    corr_matrix1 = X_train.corr()
    corr_matrix2 = df_generated.corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    sns.heatmap(corr_matrix1, annot=True, cmap='coolwarm', ax=axes[0])
    axes[0].set_title('Heatmap 1')

    sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Heatmap 2')

    plt.tight_layout()
    plt.savefig(FILE_NAME + 'HeatMaps')
    #plt.show()

    # Calculate mean and standard deviation of original and synthetic datasets
    common_columns = set(X_train.columns) & set(df_generated.columns)
    results = {}

    for column in common_columns:
        mean_df1 = X_train[column].mean()
        std_df1 = X_train[column].std()
        mean_df2 = df_generated[column].mean()
        std_df2 = df_generated[column].std()

        results[column] = {'Mean_df1': mean_df1, 'Std_df1': std_df1,
                           'Mean_df2': mean_df2, 'Std_df2': std_df2}

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(FILE_NAME + 'mean_std.csv', index=True)

    # Plot scatter plots
    new_index = pd.RangeIndex(start=0, stop=57, step=1)
    X_train = X_train.set_index(new_index)
    common_columns = set(X_train.columns) & set(df_generated.columns)

    for column in common_columns:
        plt.scatter(X_train.index, X_train[column], color='red', label='X_train')
        plt.scatter(df_generated.index, df_generated[column], color='blue', label='df_generated')

        plt.title(f"Scatter Plot: {column}")
        plt.xlabel("Index")
        plt.ylabel(column)

        plt.legend()
        plt.savefig(FILE_NAME + "_" + column)
        #plt.show()


    # Add back the class label
    X_train['class'] = 0.0
    df_generated['class'] = 0.0
    X_train.to_csv(FILE_NAME + 'Original_minority_data.csv', index=False)
    df_generated.to_csv(FILE_NAME + 'Synthetic_minority_data.csv', index=False)

    # Generate quality report and display it
    real_data = FILE_NAME + 'Original_minority_data.csv'
    synth_data = FILE_NAME + 'Synthetic_minority_data.csv'

    # report = QualityReport(data_source=synth_data, ref_data=real_data)
    # report.run()

    # print(report.peek())
    #run_summary.append([FILE_NAME, report.peek()['raw_score'], report.peek()['grade']])
    run_summary.append([FILE_NAME, last_epoch_loss, last_epoch_val_loss])

    print("Reports Saved")
    #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))
    return None

In [7]:
# Example usage
def runner(encoder_dense_layers, bottle_neck, decoder_dense_layers):
  #encoder_dense_layers = [10, 8]
  #bottle_neck = 5
  #decoder_dense_layers = [8, 10]
  #FILE_NAME = f"24_{'_'.join(map(str, encoder_dense_layers))}_{bottle_neck}_"
  FILE_NAME = f"L{NO_OF_COLUMNS}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"   
  report = generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers)
  #print(type(report))
  # Save the report object to a file
  # with open(FILE_NAME + 'quality_report.pickle', 'wb') as file:
  #     pickle.dump(report, file)

  #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))

In [8]:
# here goes nothin

In [9]:
encoder_dense_layers_trial = [[8], [10], [12], [14], [16], [18] ,[20]]
decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [10]:
total_iterations = len(bottle_neck_trial) * len(encoder_dense_layers_trial) * len(decoder_dense_layers_trial)
print("Total Model in Pipeline:",total_iterations)

# Print total models
for bn in bottle_neck_trial:
  for enc_layers in encoder_dense_layers_trial:
      for dec_layers in decoder_dense_layers_trial:
          print(enc_layers, bn, dec_layers)

Total Model in Pipeline: 294
[8] 8 [8, 10]
[8] 8 [10, 12]
[8] 8 [12, 14]
[8] 8 [14, 16]
[8] 8 [16, 18]
[8] 8 [18, 20]
[8] 8 [20, 22]
[10] 8 [8, 10]
[10] 8 [10, 12]
[10] 8 [12, 14]
[10] 8 [14, 16]
[10] 8 [16, 18]
[10] 8 [18, 20]
[10] 8 [20, 22]
[12] 8 [8, 10]
[12] 8 [10, 12]
[12] 8 [12, 14]
[12] 8 [14, 16]
[12] 8 [16, 18]
[12] 8 [18, 20]
[12] 8 [20, 22]
[14] 8 [8, 10]
[14] 8 [10, 12]
[14] 8 [12, 14]
[14] 8 [14, 16]
[14] 8 [16, 18]
[14] 8 [18, 20]
[14] 8 [20, 22]
[16] 8 [8, 10]
[16] 8 [10, 12]
[16] 8 [12, 14]
[16] 8 [14, 16]
[16] 8 [16, 18]
[16] 8 [18, 20]
[16] 8 [20, 22]
[18] 8 [8, 10]
[18] 8 [10, 12]
[18] 8 [12, 14]
[18] 8 [14, 16]
[18] 8 [16, 18]
[18] 8 [18, 20]
[18] 8 [20, 22]
[20] 8 [8, 10]
[20] 8 [10, 12]
[20] 8 [12, 14]
[20] 8 [14, 16]
[20] 8 [16, 18]
[20] 8 [18, 20]
[20] 8 [20, 22]
[8] 10 [8, 10]
[8] 10 [10, 12]
[8] 10 [12, 14]
[8] 10 [14, 16]
[8] 10 [16, 18]
[8] 10 [18, 20]
[8] 10 [20, 22]
[10] 10 [8, 10]
[10] 10 [10, 12]
[10] 10 [12, 14]
[10] 10 [14, 16]
[10] 10 [16, 18]
[10] 1

In [ ]:
'''
Type	Bottleneck	Model	Training Loss	Validation Loss
Single Encoder	B8	L24_E8_B8_D8_10	0.020200547	0.028830567
Single Encoder	B8	L24_E8_B8_D10_12	0.011874248	0.019396272
Single Encoder	B8	L24_E8_B8_D12_14	0.01253186	0.019103099
Single Encoder	B8	L24_E8_B8_D14_16	0.01223655	0.019295713
Single Encoder	B8	L24_E8_B8_D16_18	0.01211339	0.018377153
Single Encoder	B8	L24_E8_B8_D18_20	0.013265993	0.020994611
Single Encoder	B8	L24_E8_B8_D20_22	0.00853907	0.014011315
Single Encoder	B8	L24_E10_B8_D8_10	0.012718143	0.019321105
Single Encoder	B8	L24_E10_B8_D10_12	0.010368214	0.01593541
Single Encoder	B8	L24_E10_B8_D12_14	0.011432006	0.017218854
Single Encoder	B8	L24_E10_B8_D14_16	0.009766717	0.01575128
Single Encoder	B8	L24_E10_B8_D16_18	0.009817692	0.015481198
Single Encoder	B8	L24_E10_B8_D18_20	0.011965061	0.018287104
Single Encoder	B8	L24_E10_B8_D20_22	0.011875323	0.018307332
Single Encoder	B8	L24_E12_B8_D8_10	0.013718692	0.021404183
Single Encoder	B8	L24_E12_B8_D10_12	0.012269545	0.017864859
Single Encoder	B8	L24_E12_B8_D12_14	0.009018802	0.015577093
Single Encoder	B8	L24_E12_B8_D14_16	0.010896927	0.018036567
Single Encoder	B8	L24_E12_B8_D16_18	0.009903548	0.016008502
Single Encoder	B8	L24_E12_B8_D18_20	0.011372435	0.017447401
Single Encoder	B8	L24_E12_B8_D20_22	0.010066634	0.016859673
Single Encoder	B8	L24_E14_B8_D8_10	0.012476848	0.020905443
Single Encoder	B8	L24_E14_B8_D10_12	0.012231041	0.018148486
Single Encoder	B8	L24_E14_B8_D12_14	0.012664825	0.019880474
Single Encoder	B8	L24_E14_B8_D14_16	0.012472902	0.019517155
Single Encoder	B8	L24_E14_B8_D16_18	0.012170909	0.02227612
Single Encoder	B8	L24_E14_B8_D18_20	0.008503092	0.012786632
Single Encoder	B8	L24_E14_B8_D20_22	0.008345143	0.015485952
Single Encoder	B8	L24_E16_B8_D8_10	0.014301005	0.020970302
Single Encoder	B8	L24_E16_B8_D10_12	0.008801286	0.014217058
Single Encoder	B8	L24_E16_B8_D12_14	0.009399678	0.016716693
Single Encoder	B8	L24_E16_B8_D14_16	0.011148875	0.015808148
Single Encoder	B8	L24_E16_B8_D16_18	0.012288658	0.01858118
Single Encoder	B8	L24_E16_B8_D18_20	0.009127181	0.014884736
Single Encoder	B8	L24_E16_B8_D20_22	0.007531494	0.01295113
Single Encoder	B8	L24_E18_B8_D8_10	0.0155782	0.02506081
Single Encoder	B8	L24_E18_B8_D10_12	0.02023199	0.028943634
Single Encoder	B8	L24_E18_B8_D12_14	0.010849725	0.017286003
Single Encoder	B8	L24_E18_B8_D14_16	0.009916672	0.014295706
Single Encoder	B8	L24_E18_B8_D16_18	0.010803993	0.017130816
Single Encoder	B8	L24_E18_B8_D18_20	0.007940354	0.012923505
Single Encoder	B8	L24_E18_B8_D20_22	0.011614745	0.016876455
Single Encoder	B8	L24_E20_B8_D8_10	0.012028329	0.018264877
Single Encoder	B8	L24_E20_B8_D10_12	0.009807029	0.015032755
Single Encoder	B8	L24_E20_B8_D12_14	0.016898828	0.024366757
Single Encoder	B8	L24_E20_B8_D14_16	0.010364055	0.016082035
Single Encoder	B8	L24_E20_B8_D16_18	0.010803455	0.017023249
Single Encoder	B8	L24_E20_B8_D18_20	0.010748971	0.017255682
Single Encoder	B8	L24_E20_B8_D20_22	0.008082045	0.013844688
Single Encoder	B10	L24_E8_B10_D8_10	0.016949756	0.025608353
Single Encoder	B10	L24_E8_B10_D10_12	0.018969255	0.027292112
Single Encoder	B10	L24_E8_B10_D12_14	0.013042618	0.02082755
Single Encoder	B10	L24_E8_B10_D14_16	0.012259451	0.02025307
Single Encoder	B10	L24_E8_B10_D16_18	0.012465491	0.019255925
Single Encoder	B10	L24_E8_B10_D18_20	0.007189451	0.012663294
Single Encoder	B10	L24_E8_B10_D20_22	0.008652808	0.014990908
Single Encoder	B10	L24_E10_B10_D8_10	0.011901597	0.019154409
Single Encoder	B10	L24_E10_B10_D10_12	0.012873948	0.019845875
Single Encoder	B10	L24_E10_B10_D12_14	0.014086891	0.021501662
Single Encoder	B10	L24_E10_B10_D14_16	0.009237156	0.014656701
Single Encoder	B10	L24_E10_B10_D16_18	0.008330626	0.015138519
Single Encoder	B10	L24_E10_B10_D18_20	0.008865036	0.016259812
Single Encoder	B10	L24_E10_B10_D20_22	0.007975874	0.013555037
Single Encoder	B10	L24_E12_B10_D8_10	0.011515044	0.018645437
Single Encoder	B10	L24_E12_B10_D10_12	0.01067189	0.016243415
Single Encoder	B10	L24_E12_B10_D12_14	0.015903708	0.02511063
Single Encoder	B10	L24_E12_B10_D14_16	0.010774793	0.016563116
Single Encoder	B10	L24_E12_B10_D16_18	0.010840736	0.018882433
Single Encoder	B10	L24_E12_B10_D18_20	0.010688584	0.016403517
Single Encoder	B10	L24_E12_B10_D20_22	0.008477489	0.014018429
Single Encoder	B10	L24_E14_B10_D8_10	0.012266912	0.018363237
Single Encoder	B10	L24_E14_B10_D10_12	0.011742901	0.017011262
Single Encoder	B10	L24_E14_B10_D12_14	0.010399495	0.016724877
Single Encoder	B10	L24_E14_B10_D14_16	0.009052467	0.01522801
Single Encoder	B10	L24_E14_B10_D16_18	0.009595571	0.015699426
Single Encoder	B10	L24_E14_B10_D18_20	0.009627866	0.016989244
Single Encoder	B10	L24_E14_B10_D20_22	0.006805212	0.012023716
Single Encoder	B10	L24_E16_B10_D8_10	0.009650772	0.016296728
Single Encoder	B10	L24_E16_B10_D10_12	0.012286982	0.018721987
Single Encoder	B10	L24_E16_B10_D12_14	0.011885534	0.017430216
Single Encoder	B10	L24_E16_B10_D14_16	0.012043075	0.018177964
Single Encoder	B10	L24_E16_B10_D16_18	0.008174958	0.013772843
Single Encoder	B10	L24_E16_B10_D18_20	0.006986081	0.011876434
Single Encoder	B10	L24_E16_B10_D20_22	0.007057393	0.012193944
Single Encoder	B10	L24_E18_B10_D8_10	0.01260639	0.019853109
Single Encoder	B10	L24_E18_B10_D10_12	0.011601106	0.018165102
Single Encoder	B10	L24_E18_B10_D12_14	0.014833011	0.021553256
Single Encoder	B10	L24_E18_B10_D14_16	0.008646006	0.014045011
Single Encoder	B10	L24_E18_B10_D16_18	0.009786093	0.016530517
Single Encoder	B10	L24_E18_B10_D18_20	0.007978537	0.01413864
Single Encoder	B10	L24_E18_B10_D20_22	0.008742024	0.015015178
Single Encoder	B10	L24_E20_B10_D8_10	0.01189064	0.017956071
Single Encoder	B10	L24_E20_B10_D10_12	0.011150471	0.016938534
Single Encoder	B10	L24_E20_B10_D12_14	0.010044493	0.015766885
Single Encoder	B10	L24_E20_B10_D14_16	0.010684288	0.017532732
Single Encoder	B10	L24_E20_B10_D16_18	0.008295272	0.014483175
Single Encoder	B10	L24_E20_B10_D18_20	0.009312947	0.015244584
Single Encoder	B10	L24_E20_B10_D20_22	0.007614589	0.012555952
Single Encoder	B12	L24_E8_B14_D8_10	0.012740673	0.018963696
Single Encoder	B12	L24_E8_B14_D10_12	0.020524953	0.029733529
Single Encoder	B12	L24_E8_B14_D12_14	0.012521153	0.021090154
Single Encoder	B12	L24_E8_B14_D14_16	0.01360478	0.020601772
Single Encoder	B12	L24_E8_B14_D16_18	0.009351243	0.014459433
Single Encoder	B12	L24_E8_B14_D18_20	0.008703504	0.013648693
Single Encoder	B12	L24_E8_B14_D20_22	0.00999572	0.01508201
Single Encoder	B12	L24_E10_B14_D8_10	0.012655965	0.018748287
Single Encoder	B12	L24_E10_B14_D10_12	0.013155832	0.019439874
Single Encoder	B12	L24_E10_B14_D12_14	0.009333995	0.014436424
Single Encoder	B12	L24_E10_B14_D14_16	0.011391534	0.017776897
Single Encoder	B12	L24_E10_B14_D16_18	0.008913903	0.014509321
Single Encoder	B12	L24_E10_B14_D18_20	0.010969568	0.018331792
Single Encoder	B12	L24_E10_B14_D20_22	0.006724878	0.012893437
Single Encoder	B12	L24_E12_B14_D8_10	0.012032526	0.018387644
Single Encoder	B12	L24_E12_B14_D10_12	0.013319273	0.020419395
Single Encoder	B12	L24_E12_B14_D12_14	0.008726202	0.013556952
Single Encoder	B12	L24_E12_B14_D14_16	0.011590268	0.019192679
Single Encoder	B12	L24_E12_B14_D16_18	0.011300148	0.017731803
Single Encoder	B12	L24_E12_B14_D18_20	0.008734324	0.014091142
Single Encoder	B12	L24_E12_B14_D20_22	0.007467643	0.013575653
Single Encoder	B12	L24_E14_B14_D8_10	0.010648782	0.017643252
Single Encoder	B12	L24_E14_B14_D10_12	0.011172156	0.018564371
Single Encoder	B12	L24_E14_B14_D12_14	0.009192271	0.013783957
Single Encoder	B12	L24_E14_B14_D14_16	0.007650199	0.01363969
Single Encoder	B12	L24_E14_B14_D16_18	0.007060223	0.012913625
Single Encoder	B12	L24_E14_B14_D18_20	0.006993726	0.012602563
Single Encoder	B12	L24_E14_B14_D20_22	0.007764847	0.013654099
Single Encoder	B12	L24_E16_B14_D8_10	0.012610249	0.018500034
Single Encoder	B12	L24_E16_B14_D10_12	0.011372631	0.01922969
Single Encoder	B12	L24_E16_B14_D12_14	0.010875065	0.016999971
Single Encoder	B12	L24_E16_B14_D14_16	0.008488627	0.013638562
Single Encoder	B12	L24_E16_B14_D16_18	0.007786373	0.012874111
Single Encoder	B12	L24_E16_B14_D18_20	0.00825224	0.013048396
Single Encoder	B12	L24_E16_B14_D20_22	0.007396311	0.012988295
Single Encoder	B12	L24_E18_B14_D8_10	0.01248792	0.018702013
Single Encoder	B12	L24_E18_B14_D10_12	0.011104395	0.016935103
Single Encoder	B12	L24_E18_B14_D12_14	0.008232754	0.016014185
Single Encoder	B12	L24_E18_B14_D14_16	0.008487706	0.014234376
Single Encoder	B12	L24_E18_B14_D16_18	0.007279285	0.01370035
Single Encoder	B12	L24_E18_B14_D18_20	0.006768136	0.011207266
Single Encoder	B12	L24_E18_B14_D20_22	0.005223037	0.010498154
Single Encoder	B12	L24_E20_B14_D8_10	0.012261627	0.018556969
Single Encoder	B12	L24_E20_B14_D10_12	0.007556616	0.012247154
Single Encoder	B12	L24_E20_B14_D12_14	0.007920614	0.012229303
Single Encoder	B12	L24_E20_B14_D14_16	0.007362268	0.013757278
Single Encoder	B12	L24_E20_B14_D16_18	0.007153251	0.012212022
Single Encoder	B12	L24_E20_B14_D18_20	0.006679768	0.012414711
Single Encoder	B12	L24_E20_B14_D20_22	0.00647326	0.011433407
Single Encoder	B14	L24_E8_B14_D8_10	0.012740673	0.018963696
Single Encoder	B14	L24_E8_B14_D10_12	0.020524953	0.029733529
Single Encoder	B14	L24_E8_B14_D12_14	0.012521153	0.021090154
Single Encoder	B14	L24_E8_B14_D14_16	0.01360478	0.020601772
Single Encoder	B14	L24_E8_B14_D16_18	0.009351243	0.014459433
Single Encoder	B14	L24_E8_B14_D18_20	0.008703504	0.013648693
Single Encoder	B14	L24_E8_B14_D20_22	0.00999572	0.01508201
Single Encoder	B14	L24_E10_B14_D8_10	0.012655965	0.018748287
Single Encoder	B14	L24_E10_B14_D10_12	0.013155832	0.019439874
Single Encoder	B14	L24_E10_B14_D12_14	0.009333995	0.014436424
Single Encoder	B14	L24_E10_B14_D14_16	0.011391534	0.017776897
Single Encoder	B14	L24_E10_B14_D16_18	0.008913903	0.014509321
Single Encoder	B14	L24_E10_B14_D18_20	0.010969568	0.018331792
Single Encoder	B14	L24_E10_B14_D20_22	0.006724878	0.012893437
Single Encoder	B14	L24_E12_B14_D8_10	0.012032526	0.018387644
Single Encoder	B14	L24_E12_B14_D10_12	0.013319273	0.020419395
Single Encoder	B14	L24_E12_B14_D12_14	0.008726202	0.013556952
Single Encoder	B14	L24_E12_B14_D14_16	0.011590268	0.019192679
Single Encoder	B14	L24_E12_B14_D16_18	0.011300148	0.017731803
Single Encoder	B14	L24_E12_B14_D18_20	0.008734324	0.014091142
Single Encoder	B14	L24_E12_B14_D20_22	0.007467643	0.013575653
Single Encoder	B14	L24_E14_B14_D8_10	0.010648782	0.017643252
Single Encoder	B14	L24_E14_B14_D10_12	0.011172156	0.018564371
Single Encoder	B14	L24_E14_B14_D12_14	0.009192271	0.013783957
Single Encoder	B14	L24_E14_B14_D14_16	0.007650199	0.01363969
Single Encoder	B14	L24_E14_B14_D16_18	0.007060223	0.012913625
Single Encoder	B14	L24_E14_B14_D18_20	0.006993726	0.012602563
Single Encoder	B14	L24_E14_B14_D20_22	0.007764847	0.013654099
Single Encoder	B14	L24_E16_B14_D8_10	0.012610249	0.018500034
Single Encoder	B14	L24_E16_B14_D10_12	0.011372631	0.01922969
Single Encoder	B14	L24_E16_B14_D12_14	0.010875065	0.016999971
Single Encoder	B14	L24_E16_B14_D14_16	0.008488627	0.013638562
Single Encoder	B14	L24_E16_B14_D16_18	0.007786373	0.012874111
Single Encoder	B14	L24_E16_B14_D18_20	0.00825224	0.013048396
Single Encoder	B14	L24_E16_B14_D20_22	0.007396311	0.012988295
Single Encoder	B14	L24_E18_B14_D8_10	0.01248792	0.018702013
Single Encoder	B14	L24_E18_B14_D10_12	0.011104395	0.016935103
Single Encoder	B14	L24_E18_B14_D12_14	0.008232754	0.016014185
Single Encoder	B14	L24_E18_B14_D14_16	0.008487706	0.014234376
Single Encoder	B14	L24_E18_B14_D16_18	0.007279285	0.01370035
Single Encoder	B14	L24_E18_B14_D18_20	0.006768136	0.011207266
Single Encoder	B14	L24_E18_B14_D20_22	0.005223037	0.010498154
Single Encoder	B14	L24_E20_B14_D8_10	0.012261627	0.018556969
Single Encoder	B14	L24_E20_B14_D10_12	0.007556616	0.012247154
Single Encoder	B14	L24_E20_B14_D12_14	0.007920614	0.012229303
Single Encoder	B14	L24_E20_B14_D14_16	0.007362268	0.013757278
Single Encoder	B14	L24_E20_B14_D16_18	0.007153251	0.012212022
Single Encoder	B14	L24_E20_B14_D18_20	0.006679768	0.012414711
Single Encoder	B14	L24_E20_B14_D20_22	0.00647326	0.011433407
Single Encoder	B16	L24_E10_B16_D8_10	0.021984361	0.030835049
Single Encoder	B16	L24_E14_B16_D8_10	0.013890766	0.021129016
Single Encoder	B16	L24_E8_B16_D10_12	0.013782358	0.021706827
Single Encoder	B16	L24_E14_B16_D20_22	0.012705156	0.018760063
Single Encoder	B16	L24_E8_B16_D8_10	0.012688873	0.017921396
Single Encoder	B16	L24_E10_B16_D10_12	0.012404944	0.018704301
Single Encoder	B16	L24_E12_B16_D8_10	0.012352737	0.018989982
Single Encoder	B16	L24_E12_B16_D10_12	0.01204028	0.018769432
Single Encoder	B16	L24_E18_B16_D14_16	0.012037106	0.018632334
Single Encoder	B16	L24_E16_B16_D10_12	0.011915557	0.018089248
Single Encoder	B16	L24_E18_B16_D10_12	0.011890655	0.017925013
Single Encoder	B16	L24_E10_B16_D12_14	0.011780714	0.019034553
Single Encoder	B16	L24_E14_B16_D16_18	0.011674876	0.017863927
Single Encoder	B16	L24_E14_B16_D10_12	0.011197852	0.018148545
Single Encoder	B16	L24_E10_B16_D14_16	0.01118244	0.01733228
Single Encoder	B16	L24_E12_B16_D16_18	0.011144614	0.018041218
Single Encoder	B16	L24_E10_B16_D16_18	0.010928698	0.017316675
Single Encoder	B16	L24_E8_B16_D16_18	0.010879449	0.01796308
Single Encoder	B16	L24_E12_B16_D12_14	0.010470808	0.017008357
Single Encoder	B16	L24_E18_B16_D8_10	0.010253902	0.017202953
Single Encoder	B16	L24_E8_B16_D14_16	0.010106602	0.015690781
Single Encoder	B16	L24_E18_B16_D12_14	0.00983367	0.016292337
Single Encoder	B16	L24_E8_B16_D18_20	0.009697117	0.016598277
Single Encoder	B16	L24_E16_B16_D8_10	0.009657417	0.01488345
Single Encoder	B16	L24_E12_B16_D18_20	0.009572235	0.013868783
Single Encoder	B16	L24_E20_B16_D8_10	0.009494251	0.015075733
Single Encoder	B16	L24_E16_B16_D12_14	0.009335841	0.015393255
Single Encoder	B16	L24_E16_B16_D16_18	0.009165053	0.014850797
Single Encoder	B16	L24_E14_B16_D12_14	0.008562817	0.014119172
Single Encoder	B16	L24_E20_B16_D10_12	0.008479287	0.013376766
Single Encoder	B16	L24_E20_B16_D12_14	0.00827579	0.014391994
Single Encoder	B16	L24_E18_B16_D16_18	0.008172121	0.013568566
Single Encoder	B16	L24_E10_B16_D18_20	0.008082211	0.013543161
Single Encoder	B16	L24_E12_B16_D14_16	0.007948872	0.013168458
Single Encoder	B16	L24_E20_B16_D14_16	0.00763595	0.012591572
Single Encoder	B16	L24_E8_B16_D20_22	0.007488241	0.013654675
Single Encoder	B16	L24_E20_B16_D16_18	0.007446972	0.012189567
Single Encoder	B16	L24_E10_B16_D20_22	0.007425106	0.012781939
Single Encoder	B16	L24_E20_B16_D20_22	0.007311633	0.012852778
Single Encoder	B16	L24_E16_B16_D14_16	0.007254895	0.012686105
Single Encoder	B16	L24_E14_B16_D18_20	0.00716572	0.012285043
Single Encoder	B16	L24_E14_B16_D14_16	0.007064799	0.011339166
Single Encoder	B16	L24_E16_B16_D18_20	0.006791549	0.012412399
Single Encoder	B16	L24_E8_B16_D12_14	0.006711975	0.012664024
Single Encoder	B16	L24_E18_B16_D20_22	0.006687312	0.011746798
Single Encoder	B16	L24_E20_B16_D18_20	0.006570224	0.012289818
Single Encoder	B16	L24_E16_B16_D20_22	0.006317725	0.01113032
Single Encoder	B16	L24_E18_B16_D18_20	0.005509513	0.011702408
Single Encoder	B16	L24_E12_B16_D20_22	0.005355635	0.010422746
Single Encoder	B18	L24_E12_B18_D8_10	0.014949257	0.022217035
Single Encoder	B18	L24_E8_B18_D18_20	0.013219418	0.02103442
Single Encoder	B18	L24_E8_B18_D8_10	0.012654816	0.020097762
Single Encoder	B18	L24_E18_B18_D8_10	0.012506477	0.020226952
Single Encoder	B18	L24_E10_B18_D8_10	0.012390879	0.019398911
Single Encoder	B18	L24_E12_B18_D10_12	0.012184532	0.018066688
Single Encoder	B18	L24_E8_B18_D20_22	0.011988841	0.020420102
Single Encoder	B18	L24_E10_B18_D10_12	0.011950794	0.018922102
Single Encoder	B18	L24_E12_B18_D12_14	0.011938177	0.017552342
Single Encoder	B18	L24_E20_B18_D8_10	0.011696067	0.018572526
Single Encoder	B18	L24_E8_B18_D10_12	0.011556064	0.019518193
Single Encoder	B18	L24_E10_B18_D16_18	0.011491798	0.018319936
Single Encoder	B18	L24_E14_B18_D10_12	0.011282017	0.018585136
Single Encoder	B18	L24_E14_B18_D18_20	0.011133582	0.018693281
Single Encoder	B18	L24_E18_B18_D10_12	0.011006914	0.016345043
Single Encoder	B18	L24_E16_B18_D8_10	0.010818432	0.019105649
Single Encoder	B18	L24_E14_B18_D8_10	0.01072857	0.018477695
Single Encoder	B18	L24_E12_B18_D20_22	0.010723098	0.01691518
Single Encoder	B18	L24_E8_B18_D12_14	0.010651222	0.017238021
Single Encoder	B18	L24_E10_B18_D20_22	0.010629472	0.018382633
Single Encoder	B18	L24_E8_B18_D16_18	0.010365207	0.018171052
Single Encoder	B18	L24_E12_B18_D18_20	0.00994858	0.016753715
Single Encoder	B18	L24_E16_B18_D14_16	0.009810986	0.016740838
Single Encoder	B18	L24_E14_B18_D12_14	0.009796838	0.015983678
Single Encoder	B18	L24_E16_B18_D10_12	0.009491593	0.015401123
Single Encoder	B18	L24_E16_B18_D12_14	0.009464394	0.014926938
Single Encoder	B18	L24_E10_B18_D12_14	0.009412815	0.016477685
Single Encoder	B18	L24_E12_B18_D14_16	0.009356401	0.015615082
Single Encoder	B18	L24_E8_B18_D14_16	0.009201726	0.01471311
Single Encoder	B18	L24_E20_B18_D16_18	0.008983417	0.01525004
Single Encoder	B18	L24_E14_B18_D20_22	0.008961049	0.015790621
Single Encoder	B18	L24_E10_B18_D14_16	0.008807947	0.014398383
Single Encoder	B18	L24_E16_B18_D20_22	0.008700992	0.01474895
Single Encoder	B18	L24_E18_B18_D12_14	0.008686615	0.013487055
Single Encoder	B18	L24_E20_B18_D12_14	0.008317877	0.013212925
Single Encoder	B18	L24_E14_B18_D16_18	0.008079384	0.012883612
Single Encoder	B18	L24_E16_B18_D16_18	0.008014003	0.015461953
Single Encoder	B18	L24_E20_B18_D10_12	0.008008632	0.012949982
Single Encoder	B18	L24_E14_B18_D14_16	0.007869046	0.013459088
Single Encoder	B18	L24_E20_B18_D18_20	0.007327856	0.013842689
Single Encoder	B18	L24_E18_B18_D14_16	0.007200455	0.012886061
Single Encoder	B18	L24_E18_B18_D20_22	0.007134485	0.012785592
Single Encoder	B18	L24_E18_B18_D16_18	0.007037685	0.011473803
Single Encoder	B18	L24_E20_B18_D14_16	0.00695941	0.012820405
Single Encoder	B18	L24_E18_B18_D18_20	0.006834459	0.012424493
Single Encoder	B18	L24_E16_B18_D18_20	0.006461924	0.011961062
Single Encoder	B18	L24_E10_B18_D18_20	0.00637893	0.011203368
Single Encoder	B18	L24_E12_B18_D16_18	0.006345434	0.012017799
Single Encoder	B18	L24_E20_B18_D20_22	0.005598254	0.010297357
'''